In [1]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
reg_path = "all_registration.csv"
reg_header = ["day", "date", "time", "user_id", "action", "born", "phone", "country", "acquired_from"]
registration = pd.read_csv(reg_path, delimiter=';', names = reg_header)
registration["date"] = pd.to_datetime(registration["date"])

free_path = "all_free_tree.csv"
free_tree = pd.read_csv(free_path, delimiter=';', names =["day", "date", "user_id", "action"])
free_tree["date"] = pd.to_datetime(free_tree["date"])

super_path = "all_super_tree.csv"
super_tree = pd.read_csv(super_path, delimiter=';', names =["day", "date", "user_id", "action"])
super_tree["date"] = pd.to_datetime(super_tree["date"])

In [233]:
from datetime import timedelta
import time

start_time = time.time()

super_free = free_tree.merge(super_tree, how="outer", on=["day", "date", "user_id", "action"])
last_day = registration.date.max()
full_cohort = []
cohort_with_date = {}

for w in range(15):
    
    last_month = last_day - timedelta(days=30-w)

    reg_day = registration[registration.date == last_month][["user_id","date"]]
    cohort_row = []
    yesterday = reg_day[["user_id"]]

    for i in range(15):
        day_n = f"day_{i}_action"

        new_day = last_month + timedelta(days=i)
        action_day = super_free[super_free.date == new_day][["user_id","date"]]
        reg_day[day_n] = reg_day.user_id.isin(action_day.user_id)

        active_members = reg_day[reg_day[day_n] == True][["user_id",day_n]]
        users = reg_day[reg_day.user_id.isin(active_members.user_id)][["user_id"]]
        returned = users.merge(yesterday, how="inner", on="user_id")[["user_id"]]
        cohort_row.append(len(returned))
        yesterday = users
    
    full_cohort.append(cohort_row)
    cohort_with_date[last_month] = full_cohort[w]
    
print("Merge method took:", time.time() - start_time)

Merge method took: 4.942858934402466


In [235]:
from datetime import timedelta

import time

start_time = time.time()

super_free = free_tree.merge(super_tree, how="outer", on=["day", "date", "user_id", "action"])
last_day = registration.date.max()
full_cohort = []
cohort_dates = []

for w in range(15):
    
    last_month = last_day - timedelta(days=30-w)

    reg_day = registration[registration.date == last_month][["user_id","date"]]
    cohort_row = []
    yesterday = reg_day[["user_id"]]

    for i in range(15):
        day_n = f"day_{i}_action"

        new_day = last_month + timedelta(days=i)
        action_day = super_free[super_free.date == new_day][["user_id","date"]]
        reg_day[day_n] = reg_day.user_id.isin(action_day.user_id)

        active_members = reg_day[reg_day[day_n] == True][["user_id",day_n]]
        users = reg_day[reg_day.user_id.isin(active_members.user_id)][["user_id"]]
        returned = yesterday[yesterday["user_id"].isin(users["user_id"])]
        cohort_row.append(len(returned))
        yesterday = users
    
    full_cohort.append(cohort_row)
    cohort_dates.append(last_month)
    
print("isin method took:", time.time() - start_time)

isin method took: 5.2147135734558105


In [236]:
cohort_dates

[Timestamp('2024-06-11 00:00:00'),
 Timestamp('2024-06-12 00:00:00'),
 Timestamp('2024-06-13 00:00:00'),
 Timestamp('2024-06-14 00:00:00'),
 Timestamp('2024-06-15 00:00:00'),
 Timestamp('2024-06-16 00:00:00'),
 Timestamp('2024-06-17 00:00:00'),
 Timestamp('2024-06-18 00:00:00'),
 Timestamp('2024-06-19 00:00:00'),
 Timestamp('2024-06-20 00:00:00'),
 Timestamp('2024-06-21 00:00:00'),
 Timestamp('2024-06-22 00:00:00'),
 Timestamp('2024-06-23 00:00:00'),
 Timestamp('2024-06-24 00:00:00'),
 Timestamp('2024-06-25 00:00:00')]

In [246]:
column_names = [f"day_{i}" for i in range(15)]
cohort = pd.DataFrame(full_cohort, columns=column_names)
#cohort["date"] = cohort_dates
cohort.insert(0, "reg_date", cohort_dates)
cohort

,reg_date,day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14
0,2024-06-11,2284,1819,1045,736,661,604,587,547,490,455,399,398,346,287,294
1,2024-06-12,2342,1832,1017,683,590,550,529,500,449,434,382,366,308,273,266
2,2024-06-13,2292,1758,956,653,570,533,517,514,461,411,362,355,319,282,262
3,2024-06-14,2218,1701,1003,674,576,522,509,489,460,406,348,341,310,272,248
4,2024-06-15,1601,1247,685,496,442,387,368,378,319,300,291,263,231,207,194
5,2024-06-16,1753,1373,825,570,509,467,439,443,382,364,329,318,276,218,197
6,2024-06-17,2255,1765,1022,714,632,558,524,520,490,450,390,333,286,280,269
7,2024-06-18,2259,1737,954,654,566,513,486,482,443,393,360,347,322,281,268
8,2024-06-19,2340,1837,1068,711,631,576,549,535,464,441,403,375,339,284,266
9,2024-06-20,2535,2031,1166,788,691,662,593,574,557,520,494,470,401,338,317
